#### KoNLPy
* https://konlpy-ko.readthedocs.io/ko/v0.4.3/api/konlpy.tag/
* 꼬꼬마(Kkma), 한나눔(Hannanum), Komoran, 은전한닢프로젝트(Mecab), Twitter(OKt)모두 사용가능
* Mecab 윈도우환경 구동되지 않음, 
* 윈도우 설치시 자바클래스 호출을 위한 JPype1 모듈설치 (JAVA_HOME, PATH)
    - 맞는 .whl 설치 (pip)
    - pip install --upgrade pip
    - pip install JPype1-0.6.3-cp36-cp36m-win_amd64.whl (conda install -c conda-forge jpype1)
    - JAVA_HOME 일반적으로 bin앞 폴더까지이지만 jpype1에서는 \bin\server (jvm.dll)이 있는 폴더까지 지정
    (- PATH)
    - pip install konlpy (conda install -c conda-forge konlpy)

In [1]:
import pandas as pd
train_df = pd.read_csv('./data/ratings_train.txt', sep='\t')
train_df.head(3)


Bad key "axes_unicode_minus" on line 3 in
/home/jovyan/.config/matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


,id,document,label
0,15745720,재미있고 또 보고 싶네요.,1
1,15862701,여러분 인생에 한시간을 버리고 싶다면 이 영화를 꼭 보시기 바랍니다. 내용도 전개도...,0
2,15122150,무조건 강추. 실화가 아닌 영화라 해도 강추인데. 실화라는게 믿겨지지 않을 정도. ...,1


In [2]:
train_df['label'].value_counts()

1    75090
0    74910
Name: label, dtype: int64

In [3]:
import re 
# null -> 공백
train_df = train_df.fillna('')
# 숫자 -> 공백
train_df['document'] = train_df['document'].apply(lambda x: re.sub(r"\d+", " ", x))
print(train_df.head(3))
print(train_df.info())

test_df = pd.read_csv('ratings_test.txt', sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply(lambda x: re.sub(r"\d+", " ", x))

         id                                           document  label
0  15745720                                     재미있고 또 보고 싶네요.      1
1  15862701  여러분 인생에 한시간을 버리고 싶다면 이 영화를 꼭 보시기 바랍니다. 내용도 전개도...      0
2  15122150  무조건 강추. 실화가 아닌 영화라 해도 강추인데. 실화라는게 믿겨지지 않을 정도. ...      1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
id          150000 non-null int64
document    150000 non-null object
label       150000 non-null int64
dtypes: int64(2), object(1)
memory usage: 3.4+ MB
None


In [4]:
# TF-IDF 단어 벡터화 (문장 -> 형태소단어로 토큰화)
from konlpy.tag import Twitter
# Mecab japan -> korean 응용 not dependency java
# Twitter korean Text
# nouns : 명사 추출
# morphs : 형태소 추출
# pos : 품사 부착
tw = Twitter()
def tw_tokenizer(text):
    tokens_ko = tw.morphs(text) # 문장(text) -> 형태소단어로 토큰화 list 반환
    #print(tokens_ko)
    return tokens_ko
#tw_tokenizer(u'아버지가 방에 들어 가신다')


/opt/conda/lib/python3.7/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Twitter 객체의 morphs( ) 객체를 이용한 tokenizer를 사용. ngram_range는 (1,2) 
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

In [7]:
# Logistic Regression 을 이용하여 감성 분석 Classification 수행. 
lg_clf = LogisticRegression(random_state=0)

# Parameter C 최적화를 위해 GridSearchCV 를 이용. 
params = { 'C': [1 ,3.5, 4.5, 5.5, 10 ] }
grid_cv = GridSearchCV(lg_clf , param_grid=params , cv=3 ,scoring='accuracy', verbose=1 )
grid_cv.fit(tfidf_matrix_train , train_df['label'] )
print(grid_cv.best_params_ , round(grid_cv.best_score_,4))


Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   41.1s finished


{'C': 4.5} 0.8799


In [8]:
from sklearn.metrics import accuracy_score

# 학습 데이터를 적용한 TfidfVectorizer를 이용하여 테스트 데이터를 TF-IDF 값으로 Feature 변환함. 
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

# classifier 는 GridSearchCV에서 최적 파라미터로 학습된 classifier를 그대로 이용
best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)

print('Logistic Regression 정확도: ',accuracy_score(test_df['label'],preds))
# 4분정도 소요

Logistic Regression 정확도:  0.84206
